In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from collections import defaultdict

In [14]:
# Load and shuffle dataset
def load_and_split_dataset(file_path, train_ratio=0.7, random_seed=42):
    data = pd.read_csv(file_path).to_numpy()
    np.random.seed(random_seed)
    np.random.shuffle(data)
    train_size = int(round(len(data) * train_ratio))
    return data[:train_size], data[train_size:]

# Load dataset
FILE_PATH = 'data_banknote_authentication.csv'
train_data, test_data = load_and_split_dataset(FILE_PATH)

print("Training Data Shape:", train_data.shape)
print("Testing Data Shape:", test_data.shape)


Training Data Shape: (960, 5)
Testing Data Shape: (411, 5)


In [15]:
# Gini index for impurity calculation
def gini_index(data_points):
    labels = data_points[:, -1]
    if len(labels) == 0:
        return 0
    _, counts = np.unique(labels, return_counts=True)
    pi = counts / len(labels)
    return 1 - np.sum(np.square(pi))


In [16]:
# Tree node structure
class Node:
    def __init__(self, depth, split_feature=None, split_point=None, label=None):
        self.depth = depth
        self.split_feature = split_feature
        self.split_point = split_point
        self.label = label
        self.left = None
        self.right = None


In [17]:
# Best split finder
def find_best_split(data_points, num_splits=1000):
    num_features = data_points.shape[1] - 1
    best_feature, best_split_point = None, None
    best_loss = float('inf')
    best_left, best_right = None, None

    for i in range(num_features):
        feature_vals = data_points[:, i]
        min_val, max_val = np.min(feature_vals), np.max(feature_vals)
        if max_val == min_val:
            continue
        splits = np.linspace(min_val, max_val, num_splits + 2)[1:-1]
        for split_point in splits:
            left = data_points[feature_vals <= split_point]
            right = data_points[feature_vals > split_point]
            if len(left) == 0 or len(right) == 0:
                continue
            loss = len(left) * gini_index(left) + len(right) * gini_index(right)
            if loss < best_loss:
                best_loss = loss
                best_feature, best_split_point = i, split_point
                best_left, best_right = left, right

    return best_feature, best_split_point, best_left, best_right

# Stopping criteria
def should_stop(data_points, depth, max_depth, min_samples):
    return len(data_points) < min_samples or depth >= max_depth


In [18]:
# Recursive tree builder
def build_tree(data_points, depth=0, max_depth=5, min_samples=10):
    if should_stop(data_points, depth, max_depth, min_samples):
        labels = data_points[:, -1]
        majority_label = np.bincount(labels.astype(int)).argmax()
        return Node(depth, label=majority_label)

    best_feature, best_split, left, right = find_best_split(data_points)
    if best_feature is None:
        labels = data_points[:, -1]
        majority_label = np.bincount(labels.astype(int)).argmax()
        return Node(depth, label=majority_label)

    node = Node(depth, split_feature=best_feature, split_point=best_split)
    node.left = build_tree(left, depth + 1, max_depth, min_samples)
    node.right = build_tree(right, depth + 1, max_depth, min_samples)
    return node

# Build the tree
root = build_tree(train_data, max_depth=5, min_samples=10)


In [19]:
# Predict a single sample
def predict(node, data_point):
    if node.label is not None:
        return node.label
    if data_point[node.split_feature] <= node.split_point:
        return predict(node.left, data_point)
    else:
        return predict(node.right, data_point)

# Predict all samples
def predict_all(root, data):
    return [predict(root, x) for x in data]


In [20]:
# Evaluate model
def evaluate_model(model, X, y, name="Set"):
    preds = predict_all(model, X)
    acc = accuracy_score(y, preds)
    print(f"\n {name} Accuracy: {acc * 100:.2f}%")
    print("Confusion Matrix:\n", confusion_matrix(y, preds))
    print("Classification Report:\n", classification_report(y, preds))
    return acc

# Evaluate on train and test
evaluate_model(root, train_data[:, :-1], train_data[:, -1], name="Train")
evaluate_model(root, test_data[:, :-1], test_data[:, -1], name="Test")



 Train Accuracy: 98.02%
Confusion Matrix:
 [[543   1]
 [ 18 398]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98       544
         1.0       1.00      0.96      0.98       416

    accuracy                           0.98       960
   macro avg       0.98      0.98      0.98       960
weighted avg       0.98      0.98      0.98       960


 Test Accuracy: 96.35%
Confusion Matrix:
 [[214   3]
 [ 12 182]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       217
         1.0       0.98      0.94      0.96       194

    accuracy                           0.96       411
   macro avg       0.97      0.96      0.96       411
weighted avg       0.96      0.96      0.96       411



0.9635036496350365

In [21]:
# Pretty print the tree
def print_tree(node, indent=""):
    if node.label is not None:
        print(indent + f"Leaf: Predict {node.label}")
        return
    print(indent + f"[X{node.split_feature} <= {node.split_point:.4f}]")
    print(indent + "--> True:")
    print_tree(node.left, indent + "    ")
    print(indent + "--> False:")
    print_tree(node.right, indent + "    ")

# Print the tree
print("\n Decision Tree Structure:")
print_tree(root)



 Decision Tree Structure:
[X0 <= 0.3111]
--> True:
    [X1 <= 7.5052]
    --> True:
        [X0 <= -0.4728]
        --> True:
            [X2 <= 6.0996]
            --> True:
                [X0 <= -6.7324]
                --> True:
                    Leaf: Predict 1
                --> False:
                    Leaf: Predict 1
            --> False:
                [X1 <= -4.7657]
                --> True:
                    Leaf: Predict 1
                --> False:
                    Leaf: Predict 0
        --> False:
            [X2 <= 0.0986]
            --> True:
                [X1 <= 5.0155]
                --> True:
                    Leaf: Predict 1
                --> False:
                    Leaf: Predict 0
            --> False:
                [X3 <= 0.7254]
                --> True:
                    Leaf: Predict 0
                --> False:
                    Leaf: Predict 1
    --> False:
        [X0 <= -5.1636]
        --> True:
            [X0 <= -7.0402]

In [22]:
# Count how many times each feature was split on
def count_feature_splits(node, counter):
    if node.label is not None:
        return
    counter[node.split_feature] += 1
    count_feature_splits(node.left, counter)
    count_feature_splits(node.right, counter)

# Print feature importance
feature_counter = defaultdict(int)
count_feature_splits(root, feature_counter)
print("\nFeature Importance (by split frequency):")
for feat, count in sorted(feature_counter.items()):
    print(f"Feature {feat}: {count} splits")



Feature Importance (by split frequency):
Feature 0: 15 splits
Feature 1: 3 splits
Feature 2: 5 splits
Feature 3: 2 splits
